# Integrating machine breakdowns in the environment

- To manage the machine breakdowns within the environmnet, i have implemented the`` _check_machine_breakdowns(td)`` method in the environment.

- The new td["machine_breakdowns"] is a tensor with the shape [batch_size, number of machines, number of maximum breakdowns].
- The number of maximum breakdowns is currently set to 33 in the generator/environment.
- Here, the times at which breakdowns occur correspond to entries where the number of maximum breakdowns is an even number, and the duration of that breakdown is given by the subsequent entry.
- Occurence time of the first breakdown in batch x , machine y -> td["machine_breakdowns"][ x , y , 0 ]
- The duration of the  first breakdown in batch x , machine y -> td["machine_breakdowns"][ x , y , 1 ]
- Meaning
    - Breakdown occurence times td["machine_breakdowns"][batch_idx , machine_idx , x ] , where x= 0,2,4,6,8,....32
    - Breakdown duration times td["machine_breakdowns"][batch_idx , machine_idx , y ] , where x= 1,3,5,7,9,.....33




### Documentation of the ``_check_machine_breakdowns(td)`` method

- INPUT: td
    - this method takes a tensordict representing the state of the environment as an input and follows following process

- Our main logic in here is that, we iterate over each machine in each batch
    - 1. we firstly get the breakdowns of the machine
      2. then we check all the machine breakdowns if this machine has a breakdown at the current time
      3. If machine has a breakdown
            - Then we mark machine busy_until the from this time till machine is repaired, therefore machine can not process any operation
            - Then we give the tensordict with the updated busy_until enties back

- Since, according to our assomptions a job cannot be preempted or interrupted,
- we need to check at each decision step whether a job has a machine breakdown or not

- In case a machine has a breakdown right at the beginning, we check machine breakdowns in _reset()

# Manually chekcing if the ``_check_machine_breakdowns()`` method correct functions

## 1. Implementing the environment

In [1]:
from rl4co.envs.scheduling.djssp.env import DJSSPEnv
from rl4co.models import L2DPolicy, L2DModel
from rl4co.utils import RL4COTrainer
import gc
from rl4co.envs import JSSPEnv
from rl4co.models.zoo.l2d.model import L2DPPOModel
from rl4co.models.zoo.l2d.policy import L2DPolicy4PPO
from torch.utils.data import DataLoader
import json
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, clear_output
import time
import networkx as nx
import matplotlib.pyplot as plt
from rl4co.envs import FJSPEnv
from rl4co.models.zoo.l2d import L2DModel
from rl4co.models.zoo.l2d.policy import L2DPolicy
from rl4co.models.zoo.l2d.decoder import L2DDecoder
from rl4co.models.nn.graph.hgnn import HetGNNEncoder
from rl4co.utils.trainer import RL4COTrainer
generator_params = {
"num_jobs" : 6 ,
"num_machines": 6 ,
"min_processing_time": 1 ,
"max_processing_time": 99 ,
"mtbf" : 17 ,
"mttr" : 4
}
env = DJSSPEnv(generator_params=generator_params,
_torchrl_mode=True,
stepwise_reward=True)

C:\Users\soner\anaconda3\envs\reinforce\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Reset the environment and get the generated machine_breakdowns

In [2]:
td = env.reset(batch_size=[1])
td["machine_breakdowns"][0]

tensor([[4.6538e+01, 5.0296e+00, 6.3100e+01, 1.0097e+01, 1.3300e+02, 6.9801e+00,
         2.7412e+02, 1.0043e+00, 5.5139e+02, 9.9975e+00, 1.1137e+03, 7.5229e-02,
         2.2202e+03, 8.8703e-01, 4.4504e+03, 3.7722e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [8.9034e+00, 3.2588e+00, 1.3309e+01, 3.5548e+00, 3.1885e+01, 3.2662e+00,
         7.1873e+01, 2.8204e+00, 1.4057e+02, 3.5399e-01, 2.8055e+02, 9.2605e+00,
         5.7872e+02, 3.7138e-01, 1.1748e+03, 2.7541e+00, 2.3613e+03, 7.6855e+00,
         4.7021e+03, 1.2679e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.8262e+01, 4.8151e-01, 6.3443e+01, 6.5475e+00, 1.5014e+02, 2.4163e+00,
         2.5974e+

In [9]:
print(td["machine_breakdowns"][0,0,0])

tensor(46.5375)


### Machine 0 has its first breakdown at time 46.5375. However since the we do not check the machine breakdowns yet, our tensordict entry which indicates if machines are busy (td["busy_until"]) should be full with the 0's

In [10]:
td["busy_until"]

tensor([[0., 0., 0., 0., 0., 0.]])

## 3. After that to ensure our method works correctly
    We manually advance the time to the first machine breakdown occurence time and overwrite the tensordict

In [11]:
# advance the time
td["time"] = torch.Tensor([46.53752517700195])

# check the machine breakdowns and overwrite the tensordict
td = env._check_machine_breakdowns(td)

### Now, in td["busy_until"] entry -> machine 0 must be shown as busy until it is being repaired

In [12]:
print(f"Machine 0 breakdown occurence time { td["machine_breakdowns"][0,0,0]}")
print(f"Machine 0 breakdown duration time { td["machine_breakdowns"][0,0,1]}")
td["busy_until"]

Machine 0 breakdown occurence time 46.53752517700195
Machine 0 breakdown duration time 5.029618263244629


tensor([[51.5671,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])